In [1]:
import numpy as np
import pandas as pd

In [2]:
def custom_to_datetime(date):
    # If the time is 24, set it to 0 and increment day by 1
    minutes = date[0:2]
    if ':' in minutes:
        return pd.to_datetime(date, format = '%M:%S')
    else:
        minutes = int(minutes)
        if minutes >= 60:
            newDate = str(minutes - 60) + date[2:]
            return pd.to_datetime(newDate, format = '%M:%S') + pd.Timedelta(hours=1)
        else:
            return pd.to_datetime(date, format = '%M:%S')


In [3]:
def f(row):
    if row['timeOnIce'] > 3600 and row['decision'] == 'L':
        val = 1
    else:
        val = 0
    return val

In [4]:

goalie_stats = pd.read_csv('TEAM_122_DVA/DataEngineering/db_tables/goalie_stats_by_game.csv').sort_values(by=['player_id', 'season', 'game_id']).reset_index(drop=True)

# player_stats_by_game['skaterStatsevenTimeOnIce'] = pd.to_datetime(player_stats_by_game['skaterStatsevenTimeOnIce'], format = "%M:%S")
# player_stats_by_game['skaterStatsevenTimeOnIce'] = player_stats_by_game['skaterStatsevenTimeOnIce'].dt.minute * 60 + player_stats_by_game['skaterStatsevenTimeOnIce'].dt.second

goalie_stats['timeOnIce'] = goalie_stats['timeOnIce'].apply(custom_to_datetime) 
goalie_stats['timeOnIce'] = goalie_stats['timeOnIce'].dt.hour * 3600 + goalie_stats['timeOnIce'].dt.minute * 60 + goalie_stats['timeOnIce'].dt.second
goalie_stats['ot_loss'] = goalie_stats.apply(f, axis = 1)
goalie_stats.head(10)

,player_id,game_id,season,homeOrAway,timeOnIce,assists,goals,pim,shots,saves,...,powerPlayShotsAgainst,decision,savePercentage,powerPlaySavePercentage,shortHandedSavePercentage,evenStrengthSavePercentage,goals_against,shutout,win,ot_loss
0,8466141,20,2015,H,3600,0,0,0,39,38,...,8,W,97.435897,87.500000,100.0,100.000000,1,0,1,0
1,8466141,35,2015,A,3531,0,0,0,33,32,...,7,L,96.969697,100.000000,100.0,95.833333,1,0,0,0
2,8466141,55,2015,H,3600,0,0,0,26,24,...,5,W,92.307692,80.000000,NaN,95.238095,2,0,1,0
3,8466141,68,2015,H,3538,0,0,0,36,33,...,2,L,91.666667,50.000000,NaN,94.117647,3,0,0,0
4,8466141,83,2015,A,3681,0,0,0,35,32,...,6,L,91.428571,66.666667,100.0,96.428571,3,0,0,1
5,8466141,97,2015,A,3581,0,0,0,27,24,...,7,L,88.888889,71.428571,NaN,95.000000,3,0,0,0
6,8466141,128,2015,H,3600,1,0,0,32,31,...,11,W,96.875000,100.000000,NaN,95.238095,1,0,1,0
7,8466141,150,2015,H,3361,0,0,0,30,27,...,4,L,90.000000,50.000000,100.0,96.000000,3,0,0,0
8,8466141,184,2015,A,3900,0,0,2,39,37,...,9,L,94.871795,88.888889,100.0,96.551724,2,0,0,1
9,8466141,201,2015,A,3600,0,0,0,35,31,...,2,L,88.571429,50.000000,100.0,90.625000,4,0,0,0


In [6]:
categories = ['saves', 'goals_against', 'save_percent', 
              'shots_against', 'wins', 'shutouts', 'ot_loss', 'es_shots', 
              'es_save_percent', 'pp_shots', 'pp_save_percent', 'sh_shots', 'sh_save_percent']


periods = [1, 5, 10, 20]

agg_cols = {
    'saves' : 'saves',
    'goals_against' : 'goals_against', 
    'save_percent' : 'savePercentage', 
    'shots_against' : 'shots', 
    'wins' : 'win', 
    'shutouts' : 'shutout', 
    'ot_loss' : 'ot_loss',
    'es_shots' : 'evenShotsAgainst', 
    'es_save_percent' : 'evenStrengthSavePercentage', 
    'pp_shots' : 'powerPlayShotsAgainst', 
    'pp_save_percent' : 'powerPlaySavePercentage', 
    'sh_shots' : 'shortHandedShotsAgainst', 
    'sh_save_percent' : 'shortHandedSavePercentage',
}

In [7]:
start_index = 0
season_index = 0

rows = []

for i in range(0, len(goalie_stats)):
    if (goalie_stats.iloc[start_index, 0] != goalie_stats.iloc[i, 0]):
        start_index = i
    
    if (goalie_stats.iloc[season_index, 2] != goalie_stats.iloc[i, 2]):
        season_index = i
        print(season_index)
    
    game_history = i - start_index
    season = goalie_stats.iloc[i, 2]
    
    dict1 = {}
    
    dict1['player_id'] = goalie_stats.iloc[i, 0]
    dict1['game_id'] = goalie_stats.iloc[i, 1]
    dict1['season'] = season
    dict1['homeOrAway'] = goalie_stats.iloc[i, 3]
    
    for category in categories:
        for period in periods:
            if game_history >= period:
                dict1[category + "_" + str(period)] = goalie_stats[agg_cols[category]].iloc[(i-period):i].mean()
            else:
                dict1[category + "_" + str(period)] = None
                
        if season_index == i:
            dict1[category + "_season"] = None
        else:
            dict1[category + "_season"] = goalie_stats[agg_cols[category]].iloc[season_index:i].mean()
    
    dict1['dfs_points'] = (6 * goalie_stats[agg_cols['wins']].iloc[i]) + (0.7 * goalie_stats[agg_cols['saves']].iloc[i]) + (-3.5 * goalie_stats[agg_cols['goals_against']].iloc[i]) + (4 * goalie_stats[agg_cols['shutouts']].iloc[i]) + (2 * (goalie_stats[agg_cols['ot_loss']].iloc[i]))

    if goalie_stats[agg_cols['saves']].iloc[i] >= 35:
        dict1['dfs_points'] += 3
    
    rows.append(dict1)


62
102
135
178
238
278
334
384
418
469
523
548
568
591
592
596
661
718
780
832
862
863
923
931
934
966
1021
1076
1118
1157
1168
1211
1270
1305
1323
1344
1362
1395
1413
1465
1526
1568
1601
1659
1697
1742
1803
1852
1910
1965
1993
2032
2072
2109
2135
2194
2249
2276
2312
2340
2393
2433
2464
2506
2555
2596
2622
2653
2680
2699
2712
2779
2844
2902
2969
2999
3057
3117
3156
3182
3195
3198
3212
3253
3304
3330
3373
3404
3450
3466
3496
3537
3567
3633
3694
3751
3807
3843
3856
3918
3965
4031
4089
4153
4219
4271
4317
4358
4391
4399
4417
4485
4502
4564
4610
4652
4713
4752
4805
4851
4895
4908
4910
4937
4958
4996
5045
5081
5114
5133
5187
5246
5258
5272
5279
5284
5324
5367
5410
5446
5471
5487
5493
5532
5571
5606
5641
5687
5701
5714
5771
5795
5846
5894
5939
5971
5999
6021
6028
6029
6034
6035
6061
6071
6100
6132
6174
6182
6230
6267
6290
6307
6340
6366
6424
6484
6527
6574
6635
6692
6738
6762
6792
6820
6823
6832
6848
6914
6977
7030
7089
7137
7146
7152
7153
7237
7302
7359
7420
7461
7516
7554
7583
7609
7635
76

In [8]:
df = pd.DataFrame(rows)

In [10]:
df.head(11)

,player_id,game_id,season,homeOrAway,saves_1,saves_5,saves_10,saves_20,saves_season,goals_against_1,...,sh_shots_5,sh_shots_10,sh_shots_20,sh_shots_season,sh_save_percent_1,sh_save_percent_5,sh_save_percent_10,sh_save_percent_20,sh_save_percent_season,dfs_points
0,8466141,20,2015,H,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32.1
1,8466141,35,2015,A,38.0,NaN,NaN,NaN,38.000000,1.0,...,NaN,NaN,NaN,4.000000,100.0,NaN,NaN,NaN,100.0,18.9
2,8466141,55,2015,H,32.0,NaN,NaN,NaN,35.000000,1.0,...,NaN,NaN,NaN,3.000000,100.0,NaN,NaN,NaN,100.0,15.8
3,8466141,68,2015,H,24.0,NaN,NaN,NaN,31.333333,2.0,...,NaN,NaN,NaN,2.000000,NaN,NaN,NaN,NaN,100.0,12.6
4,8466141,83,2015,A,33.0,NaN,NaN,NaN,31.750000,3.0,...,NaN,NaN,NaN,1.500000,NaN,NaN,NaN,NaN,100.0,13.9
5,8466141,97,2015,A,32.0,31.8,NaN,NaN,31.800000,3.0,...,1.4,NaN,NaN,1.400000,100.0,100.0,NaN,NaN,100.0,6.3
6,8466141,128,2015,H,24.0,29.0,NaN,NaN,30.500000,3.0,...,0.6,NaN,NaN,1.166667,NaN,100.0,NaN,NaN,100.0,24.2
7,8466141,150,2015,H,31.0,28.8,NaN,NaN,30.571429,1.0,...,0.2,NaN,NaN,1.000000,NaN,100.0,NaN,NaN,100.0,8.4
8,8466141,184,2015,A,27.0,29.4,NaN,NaN,30.125000,3.0,...,0.4,NaN,NaN,1.000000,100.0,100.0,NaN,NaN,100.0,23.9
9,8466141,201,2015,A,37.0,30.2,NaN,NaN,30.888889,2.0,...,0.6,NaN,NaN,1.000000,100.0,100.0,NaN,NaN,100.0,7.7


In [11]:
df.to_csv('goalie_feature_matrix.csv')